In [ ]:
import os
import csv

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "" 

In [ ]:
from google.cloud import bigquery

In [ ]:
client = bigquery.Client()

In [ ]:
!echo $GOOGLE_APPLICATION_CREDENTIALS

/content/maximal-centaur-120203-f1fd85adc57c.json


In [ ]:
query = """
    SELECT Id FROM `sotorrent-org.2020_12_31.Tags`
    WHERE TagName = "python"
    LIMIT 20
"""
query_job = client.query(query)  # Make an API request.

print("The query data:")
for row in query_job:
  print(row)

The query data:
Row((16,), {'Id': 0})


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def get_tag_id(tag_name: str):
  query = f"""
      SELECT Id FROM `sotorrent-org.2020_12_31.Tags`
      WHERE TagName = "{tag_name}"
      LIMIT 1
  """
  query_job = client.query(query)  # Make an API request.

  for row in query_job:
    return row[0]


def get_overlapping_tags(tag_id: int, num_tags: int = 200):
  query = f"""
    WITH newIds AS (
      WITH ids AS 
      (SELECT PostId FROM `sotorrent-org.2020_12_31.PostTags`
      WHERE TagId = {tag_id})
      SELECT TagId, COUNT(*) AS count FROM `sotorrent-org.2020_12_31.PostTags`
      JOIN ids
        ON ids.PostId = `sotorrent-org.2020_12_31.PostTags`.PostId
      GROUP BY TagId)
      
    SELECT * FROM `sotorrent-org.2020_12_31.Tags`
    JOIN newIds
      ON newIds.TagId = `sotorrent-org.2020_12_31.Tags`.Id
    ORDER BY newIds.count DESC
    LIMIT {num_tags};
  """

  query_job = client.query(query)
  data = []
  for row in query_job:
    data.append(row)
  return data

def get_post_by_id(post_id):
  query = f"""
    SELECT Tags FROM `sotorrent-org.2020_12_31.Posts`
    WHERE Id = {post_id}
  """
  query_job = client.query(query)
  data = []
  for row in query_job:
    print(row)


def download_relevant_posts(relevant_tag_ids, irrelevant_tag_ids):
  relevant_tag_str = ' '.join([f'OR {tag} IN UNNEST(ids.arr)' for tag in relevant_tag_ids])
  irrelevant_tag_str = ' '.join([f'AND {tag} NOT IN UNNEST(ids.arr)' for tag in irrelevant_tag_ids])
  query = f"""
        WITH proper_ids AS (
          WITH ids AS (
            SELECT PostId, ARRAY_AGG(TagId) AS arr 
            FROM `sotorrent-org.2020_12_31.PostTags`
            GROUP BY PostId
          )  
          SELECT PostId 
          FROM ids 
          WHERE (16 IN UNNEST(ids.arr) {relevant_tag_str})
          {irrelevant_tag_str}
        )

        SELECT Id, Title, Tags, Body, AcceptedAnswerId, Score, ViewCount FROM `sotorrent-org.2020_12_31.Posts`
        JOIN proper_ids
          ON proper_ids.PostId = `sotorrent-org.2020_12_31.Posts`.Id
        WHERE `sotorrent-org.2020_12_31.Posts`.PostTypeId = 1
        ORDER BY `sotorrent-org.2020_12_31.Posts`.Score DESC
        LIMIT 100000;
  """
  query_job = client.query(query)
  data = []
  for i, row in enumerate(query_job):
    if i % 100_000 == 0:
      print(i)
    data.append(list(row))

  save_data('questions_small.csv', data)

def create_table_relevant_questions(relevant_tag_ids, irrelevant_tag_ids):
  relevant_tag_str = ' '.join([f'OR {tag} IN UNNEST(ids.arr)' for tag in relevant_tag_ids])
  irrelevant_tag_str = ' '.join([f'AND {tag} NOT IN UNNEST(ids.arr)' for tag in irrelevant_tag_ids])
  query = f"""
        CREATE TABLE
        maximal-centaur-120203.stack_overflow_ir.large_questions (
          Id INTEGER, 
          Title STRING, 
          Tags STRING, 
          Body STRING, 
          AcceptedAnswerId INTEGER, 
          Score INTEGER, 
          ViewCount INTEGER
        ) AS 
        WITH proper_ids AS (
          WITH ids AS (
            SELECT PostId, ARRAY_AGG(TagId) AS arr 
            FROM `sotorrent-org.2020_12_31.PostTags`
            GROUP BY PostId
          )  
          SELECT PostId 
          FROM ids 
          WHERE (16 IN UNNEST(ids.arr) {relevant_tag_str})
          {irrelevant_tag_str}
        )
        SELECT Id, Title, Tags, Body, AcceptedAnswerId, Score, ViewCount FROM `sotorrent-org.2020_12_31.Posts`
        JOIN proper_ids
          ON proper_ids.PostId = `sotorrent-org.2020_12_31.Posts`.Id
        WHERE `sotorrent-org.2020_12_31.Posts`.PostTypeId = 1
        ORDER BY `sotorrent-org.2020_12_31.Posts`.Score DESC;
  """
  query_job = client.query(query)
  for i, row in enumerate(query_job):
    if i % 100_000 == 0:
      print(i)
    print(row)


def create_table_relevant_answers():
  query = f"""
        CREATE TABLE
        maximal-centaur-120203.stack_overflow_ir.large_answers (
          Id INTEGER, 
          QuestionId INTEGER,
          Body STRING, 
          Score INTEGER, 
        ) AS 
        SELECT `sotorrent-org.2020_12_31.Posts`.Id, `sotorrent-org.2020_12_31.Posts`.ParentId, `sotorrent-org.2020_12_31.Posts`.Body, `sotorrent-org.2020_12_31.Posts`.Score FROM `sotorrent-org.2020_12_31.Posts`
        JOIN `maximal-centaur-120203.stack_overflow_ir.large_questions`
          ON `maximal-centaur-120203.stack_overflow_ir.large_questions`.Id = `sotorrent-org.2020_12_31.Posts`.ParentId
        ORDER BY `sotorrent-org.2020_12_31.Posts`.Score DESC;
  """
  query_job = client.query(query)
  for i, row in enumerate(query_job):
    if i % 100_000 == 0:
      print(i)
    print(row)

def download_answers():
  query = f"""
    SELECT * FROM `maximal-centaur-120203.stack_overflow_ir.large_answers`;
  """
  query_job = client.query(query)
  data = []
  for i, row in enumerate(query_job):
    if i % 100_000 == 0:
      print(i)
    data.append(list(row))
  save_data('large_answers.csv', data)


def download_questions():
  query = f"""
    SELECT * FROM `maximal-centaur-120203.stack_overflow_ir.large_questions`;
  """
  query_job = client.query(query)
  data = []
  for i, row in enumerate(query_job):
    if i % 100_000 == 0:
      print(i)
    data.append(list(row))
  save_data('large_questions.csv', data)

def load_answer_ids(path):
  ids = []
  with open(path) as f:
    reader = csv.reader(f)
    for row in reader:
      ids.append(row[0])
  return ids

def save_data(filename, data):
  with open(f'/content/drive/My Drive/CS 685/Final Project/Data/{filename}', 'w') as f:
    writer = csv.writer(f)
    for i, row in enumerate(data):
      if i % 1_000 == 0:
        print(i, len(data))
      writer.writerow(row)

In [ ]:
dataset_id = "{}.stack_overflow_ir".format(client.project)
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
dataset = client.create_dataset(dataset)
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Created dataset maximal-centaur-120203.stack_overflow_ir


In [ ]:
download_answers()
download_questions()

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
0 2273845
1000 2273845
2000 2273845
3000 2273845
4000 2273845
5000 2273845
6000 2273845
7000 2273845
8000 2273845
9000 2273845
10000 2273845
11000 2273845
12000 2273845
13000 2273845
14000 2273845
15000 2273845
16000 2273845
17000 2273845
18000 2273845
19000 2273845
20000 2273845
21000 2273845
22000 2273845
23000 2273845
24000 2273845
25000 2273845
26000 2273845
27000 2273845
28000 2273845
29000 2273845
30000 2273845
31000 2273845
32000 2273845
33000 2273845
34000 2273845
35000 2273845
36000 2273845
37000 2273845
38000 2273845
39000 2273845
40000 2273845
41000 2273845
42000 2273845
43000 2273845
44000 2273845
45000 2273845
46000 2273845
47000 2273845
48000 2273845
49000 2273845
50000 2273845
51000 2273845
52000 2273845
53000 2273845
54000 2273845
55000 2273845
56000 2273845
57000 2273845
58000 2273845
59000 2273845
6000

dataset name:

```
maximal-centaur-120203.stack_overflow_ir
```

In [ ]:
currated_list = ['matplotlib', 'pandas', 'numpy', 'pytorch', 
                 'seaborn', 'jupyter-notebook', 'ipython', 'python-3.x', 'nlp', 'machine-learning']

currated_ids = [7979, 67719, 4190, 124253, 99284, 116342, 14670, 60010, 2370, 5990]

currated_irrelevant_list =['django', 'flask', 'javascript', 
                           'django-rest-framework', 'pyqt5', 'jinja2']

# currated_irrelevant_ids = [get_tag_id(tag) for tag in currated_irrelevant_list]
currated_irrelevant_ids = [243, 54712, 3, 81949, 94814, 20674]

In [ ]:
create_table_relevant_questions(currated_ids, currated_irrelevant_ids)

In [ ]:
answer_ids = load_answer_ids('/content/drive/My Drive/CS 685/Final Project/Data/questions_small.csv')
download_answers(answer_ids)

In [ ]:
create_table_relevant_answers()

In [ ]:
get_tag_id("machine-learning")

The query data:


5990

In [ ]:
get_post_by_id(4462952)

Row(('<python><debugging><winpdb>',), {'Tags': 0})


In [ ]:
print(currated_irrelevant_ids)

[243, 54712, 3, 81949, 94814, 20674]


In [ ]:
# Selects posts based on whether they have a certain tag or not
query = """
      WITH proper_ids AS (
        WITH ids AS (
          SELECT PostId, ARRAY_AGG(TagId) AS arr 
          FROM `sotorrent-org.2020_12_31.PostTags`
          GROUP BY PostId
        )  
        SELECT PostId 
        FROM ids 
        WHERE 16 IN UNNEST(ids.arr) 
        AND 243 NOT IN UNNEST(ids.arr) 
        AND 3 NOT IN UNNEST(ids.arr) 
      )

      SELECT Title, Tags, Body FROM `sotorrent-org.2020_12_31.Posts`
      JOIN proper_ids
        ON proper_ids.PostId = `sotorrent-org.2020_12_31.Posts`.Id
      WHERE `sotorrent-org.2020_12_31.Posts`.PostTypeId = 1
      LIMIT 200;
"""
query_job = client.query(query)
data = []
for row in query_job:
  print(row)

Row(("How to use external python modules such as the module 'requests' when running executable python scripts such as .command files?", '<python><python-3.x><python-requests><command><executable>', "<p>I made a python script into an executable file by turning it into a .command file and adding the #!/usr/bin/env python3 on the first line of the file. </p>\n\n<p>When I double click to run the file, the file does run on the terminal, however the installed modules do not seem to be linked to the file. </p>\n\n<p>import requests\nModuleNotFoundError: No module named 'requests'</p>\n\n<p>Is there a way to use external module such as the 'requests' module when running an executable form of a python script?</p>\n"), {'Title': 0, 'Tags': 1, 'Body': 2})
Row(('pandas How to drop the whole row if any specific columns contains a specific values?', '<python><pandas>', '<p>I have a dataFrame like this:\n<a href="https://i.stack.imgur.com/uPiBk.png" rel="nofollow noreferrer">enter image description h

In [ ]:
query = """
  WITH newIds AS (
    WITH ids AS 
    (SELECT PostId FROM `sotorrent-org.2020_12_31.PostTags`
    WHERE TagId = 243)
    SELECT TagId, COUNT(*) AS count FROM `sotorrent-org.2020_12_31.PostTags`
    JOIN ids
      ON ids.PostId = `sotorrent-org.2020_12_31.PostTags`.PostId
    GROUP BY TagId)
    
  SELECT * FROM `sotorrent-org.2020_12_31.Tags`
  JOIN newIds
    ON newIds.TagId = `sotorrent-org.2020_12_31.Tags`.Id
  ORDER BY newIds.count DESC
  LIMIT 200;
"""
query_job = client.query(query)
data = []
for row in query_job:
  print(row)

Row((243, 'django', 252066, 3625245, 3607538, 243, 252054), {'Id': 0, 'TagName': 1, 'Count': 2, 'ExcerptPostId': 3, 'WikiPostId': 4, 'TagId': 5, 'count_1': 6})
Row((16, 'python', 1597896, 3624965, 3607014, 16, 127555), {'Id': 0, 'TagName': 1, 'Count': 2, 'ExcerptPostId': 3, 'WikiPostId': 4, 'TagId': 5, 'count_1': 6})
Row((10742, 'django-models', 32797, 6357582, 6357581, 10742, 31157), {'Id': 0, 'TagName': 1, 'Count': 2, 'ExcerptPostId': 3, 'WikiPostId': 4, 'TagId': 5, 'count_1': 6})
Row((81949, 'django-rest-framework', 21085, 11442997, 11442996, 81949, 18166), {'Id': 0, 'TagName': 1, 'Count': 2, 'ExcerptPostId': 3, 'WikiPostId': 4, 'TagId': 5, 'count_1': 6})
Row((23506, 'django-views', 16617, 9296011, 9296010, 23506, 15804), {'Id': 0, 'TagName': 1, 'Count': 2, 'ExcerptPostId': 3, 'WikiPostId': 4, 'TagId': 5, 'count_1': 6})
Row((15108, 'django-forms', 16423, 7051214, 7051213, 15108, 15611), {'Id': 0, 'TagName': 1, 'Count': 2, 'ExcerptPostId': 3, 'WikiPostId': 4, 'TagId': 5, 'count_1': 6

In [ ]:
query = """
      WITH ids AS 
      (SELECT PostId FROM `sotorrent-org.2020_12_31.PostTags`
      WHERE TagId = 16)

      SELECT * FROM `sotorrent-org.2020_12_31.PostTags`
      JOIN ids
        ON ids.PostId = `sotorrent-org.2020_12_31.PostTags`.PostId
      LIMIT 200;
"""

# query = """
#     SELECT COUNT(PostId) FROM `sotorrent-org.2020_12_31.PostTags`
#     WHERE TagId = 16
# """

query_job = client.query(query)  # Make an API request.
# print(query_job)

# exit()
print("The query data:")
for row in query_job:
  print(row)
  # query = f"""
  #   WITH tags as SELECT TagId FROM `sotorrent-org.2020_12_31.PostTags`
  #   WHERE PostId = {row[0]}
  #   LIMIT 20;

  #   SELECT * FROM `sotorrent-org.2020_12_31.PostTags`
  #   JOIN tags
  # """
  # tags = client.query(query)
  # print([tag[0] for tag in tags])

The query data:
Row((16308989, 60010, 16308989), {'PostId': 0, 'TagId': 1, 'PostId_1': 2})
Row((42988664, 81490, 42988664), {'PostId': 0, 'TagId': 1, 'PostId_1': 2})
Row((55708654, 45137, 55708654), {'PostId': 0, 'TagId': 1, 'PostId_1': 2})
Row((56855526, 46457, 56855526), {'PostId': 0, 'TagId': 1, 'PostId_1': 2})
Row((57658703, 1834, 57658703), {'PostId': 0, 'TagId': 1, 'PostId_1': 2})
Row((4462952, 57694, 4462952), {'PostId': 0, 'TagId': 1, 'PostId_1': 2})
Row((59152894, 109163, 59152894), {'PostId': 0, 'TagId': 1, 'PostId_1': 2})
Row((48131804, 67719, 48131804), {'PostId': 0, 'TagId': 1, 'PostId_1': 2})
Row((60173633, 3496, 60173633), {'PostId': 0, 'TagId': 1, 'PostId_1': 2})
Row((34715541, 109163, 34715541), {'PostId': 0, 'TagId': 1, 'PostId_1': 2})
Row((49209163, 130245, 49209163), {'PostId': 0, 'TagId': 1, 'PostId_1': 2})
Row((49344251, 8488, 49344251), {'PostId': 0, 'TagId': 1, 'PostId_1': 2})
Row((36761902, 46457, 36761902), {'PostId': 0, 'TagId': 1, 'PostId_1': 2})
Row((248938